# Data Insights Challenge
Presented by Angus, Hiroki

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('car_prices.csv')

# **DATA CLEANING**

In [ ]:
null_counts = df.isnull().sum()
print("Null value counts in each column:")
print(null_counts)

In [ ]:
cleaned_df = df.copy()
# 'year' column
year_pattern = r'^\d{4}$'
invalid_years = df[~df['year'].astype(str).str.match(year_pattern)]
cleaned_df = cleaned_df.drop(invalid_years.index)

# 'make' & 'model' column
make_model_null = cleaned_df[cleaned_df['make'].isnull() & cleaned_df['model'].isnull()]
cleaned_df = cleaned_df.drop(make_model_null.index)
cleaned_df['make'] = cleaned_df['make'].apply(lambda x: x.capitalize())

# 'vin' column
cleaned_df = cleaned_df[cleaned_df['vin'].astype(str).str.len() == 17]

# 'color' column
color_list = ['white', 'gray', 'black', 'red', 'silver', 'blue', 'brown'
        'beige', 'purple', 'burgundy', 'gold', 'yellow', 'green',
        'charcoal', 'orange', 'off-white', 'turquoise', 'pink', 'lime']
cleaned_df.loc[~cleaned_df['color'].isin(color_list), 'color'] = None

# 'interior' column
interior_color_list = ['black', 'beige', 'tan', 'gray', 'brown',
             'burgundy', 'white', 'silver', 'off-white', 'blue'
             'red', 'yellow', 'green', 'purple', 'orange', 'gold']
cleaned_df.loc[~cleaned_df['interior'].isin(interior_color_list), 'interior'] = None

In [ ]:
# saledate column
def extract_and_convert_time(sale_str):
    # remove timezone
    try:
      sale_str = sale_str.split("(")[0].strip()
    except AttributeError as e: # in case it is nan
        return
    # Parse the string into a datetime object
    sale_datetime = datetime.strptime(sale_str, "%a %b %d %Y %H:%M:%S GMT%z")
    # Convert the time to GMT-0000 format
    gmt_time = sale_datetime.astimezone(datetime.now().astimezone().tzinfo)
    return gmt_time
def extract_weekday(sale_str):
    try:
        sale_str = sale_str.split("(")[0].strip()
    except AttributeError as e: # in case it is nan
        return
    # Parse the string into a datetime object
    sale_datetime = datetime.strptime(sale_str, "%a %b %d %Y %H:%M:%S GMT%z")
    # Extract the weekday
    weekday = sale_datetime.strftime("%A")
    return weekday

# Apply the function to the 'saletime' column
cleaned_df['saleweekday'] = cleaned_df['saledate'].apply(extract_weekday)
cleaned_df['saletime'] = cleaned_df['saledate'].apply(extract_and_convert_time)

# other columns is cleaned already! Add a final column here.
cleaned_df['profit'] = cleaned_df['sellingprice'] - cleaned_df['mmr']

In [ ]:
df = cleaned_df.copy()

In [ ]:
cleaned_df.head()

In [ ]:
# Check for duplicate values in the 'vid' column
duplicates = df[df.duplicated(subset=['vin'], keep=False)]
#print(duplicates.loc[:, ['year', 'vin']])
duplicate_pairs = duplicates.groupby('vin').apply(lambda x: x.reset_index(drop=True))
#print(duplicate_pairs.head(2))

# **DATA ANALYSIS**

## Sales & Profit

In this section, we get straight to how the car sales perform and the area for improvement.

In [ ]:
df1 = df.copy()
df1['saletime'] = pd.to_datetime(df1['saletime'], utc=True)

# Extract the date from the 'saletime' column
df1['saledate'] = df1['saletime'].dt.date
df1 = df1[df1['saledate'] > pd.to_datetime('2014-11-11').date()]
# Aggregate data by date
profit_by_date = df1.groupby('saledate')['profit'].sum().reset_index()
sales_by_date = df1['saledate'].value_counts().sort_index()

# Plotting the graph
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10), gridspec_kw={'hspace': 0.5})
ax1.plot(profit_by_date['saledate'], profit_by_date['profit'], marker='o')
ax1.set_title('Profit vs. Sale Date')
ax1.set_xlabel('Sale Date')
ax1.set_ylabel('Profit')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True)
ax2.plot(sales_by_date.index, sales_by_date.values, marker='o')
ax2.set_title('Sales Number vs. Sale Date')
ax2.set_xlabel('Sale Date')
ax2.set_ylabel('Sales Number')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True)

The graph above shows the profit over a year is a tragedy. The more cars sold, the less they earned. There's even barely sells in between March and June of the year 2015. This result indicates a lot improvement have to be done on car marketting.

In [ ]:
sales_by_weekday = df['saleweekday'].value_counts().sort_index()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_by_weekday = sales_by_weekday.reindex(weekday_order)
# Plotting the graph
plt.figure(figsize=(10, 6))
sales_by_weekday.plot(kind='bar', color='skyblue')
plt.title('Number of Sales by Weekday')
plt.xlabel('Sale Weekday')
plt.ylabel('Number of Sales')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

On the other hand, an obvious trend is shown here with a clear bell shape that people tends to purchases cars on Wednesday or days before or after. However, people are not keen to buy any cars during the weekend.

In [ ]:
no_model = df[df['model'].isnull()]['make'].value_counts(normalize=True)
print(no_model)

An Interesting fact here that BMW and Audi, especially BMW, is the only brand the will miss the model name. Something might need to be caught up here.

In [ ]:
df_positive_profit = df[df['profit'] > 0]

# Calculate total profit by manufacturer
profit_by_make = df_positive_profit.groupby('make')['profit'].sum().reset_index()
profit_by_make = profit_by_make[profit_by_make['profit'] > 1200000]
profit_by_make_sorted = profit_by_make.sort_values(by='profit', ascending=False)

# Plotting the graph
plt.figure(figsize=(10, 6))
plt.bar(profit_by_make_sorted['make'], profit_by_make_sorted['profit'], color='skyblue')
plt.title('Total Profit by Manufacturer')
plt.xlabel('Manufacturer')
plt.ylabel('Total Profit')
plt.xticks(rotation=65)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of unique car brands with total profit > 1200000
num_brands_profit_positive = profit_by_make['make'].nunique()
prop = num_brands_profit_positive / df['make'].nunique()
print("Proportion of car brands with good profit:", prop)

The graphs above shows the total profit of each maufacturer, however only maufacterer have a total profit of $1,200,000 or more will be shown on the graph. As $100000 (on month on average) is considered as a maintainable profit to the car maufacturer. The code above shows less than half of the manufacturer satisfy this condition as well, this leads us to investigate the factor that affects the profit of cars...

In [ ]:
color_proportions = df['color'].value_counts(normalize=True)
color_colors = ['black', 'white', 'silver', 'gray', 'blue', 'red', 'pink']
interior_proportions = df['interior'].value_counts(normalize=True)
interior_colors = ['black', 'gray', 'beige', 'tan', 'pink']

threshold = 0.08
# Identify colors to include in "Other" category
colors_to_combine = color_proportions[color_proportions < threshold].index
other_color_proportion = color_proportions[colors_to_combine].sum()
color_proportions_filtered = color_proportions[color_proportions >= threshold]
color_proportions_filtered['Other'] = other_color_proportion

# Identify interior colors to include in "Other" category
interiors_to_combine = interior_proportions[interior_proportions < threshold].index
other_interior_proportion = interior_proportions[interiors_to_combine].sum()
interior_proportions_filtered = interior_proportions[interior_proportions >= threshold]
interior_proportions_filtered['Other'] = other_interior_proportion

# Create figure and subplots for the pie charts
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot the color proportions pie chart
axs[0].pie(color_proportions_filtered, labels=color_proportions_filtered.index,
      autopct='%1.1f%%', startangle=90, colors=color_colors, shadow=True)
axs[0].set_title('Color Proportions')
axs[0].legend(loc="center right", bbox_to_anchor=(1.2, 0.5))

# Plot the interior proportions pie chart
axs[1].pie(interior_proportions_filtered, labels=interior_proportions_filtered.index,
      autopct='%1.1f%%', startangle=90, colors=interior_colors, shadow=True)
axs[1].set_title('Interior Proportions')
axs[1].legend(loc="center right", bbox_to_anchor=(1.2, 0.5))

# Display the pie charts
plt.tight_layout()
plt.show()

We can see that the color of the car is quite separate, revealing people don't put this as a first consideration... What could be matter then?

## Profit Difference after Resale

Story~
We found that some of the vin repeats in the data, indicating the car is being resold, so we'll investigate if reselling the car affect its value.

In [ ]:
# Check for duplicate values in the 'vid' column
duplicates = df[df.duplicated(subset=['vin'], keep=False)]
#print(duplicates.loc[:, ['year', 'vin']])
duplicate_pairs = duplicates.groupby('vin').apply(lambda x: x.reset_index(drop=True))
duplicate_pairs['profit_diff'] = duplicate_pairs.groupby(level=0)['profit'].diff()
duplicate_pairs['time_diff'] = (duplicate_pairs.groupby(level=0)['saletime'].diff().dt.total_seconds() / (24 * 3600)).astype(float)
duplicate_pairs['odom_diff'] = duplicate_pairs.groupby(level=0)['odometer'].diff()
diff = duplicate_pairs[['make', 'odom_diff', 'time_diff', 'profit_diff']].copy()
diff = diff.dropna()
diff = diff.reset_index(level=0, drop=True)

In [ ]:
average_profit_diff = diff.groupby('make')['profit_diff'].mean()
average_profit_diff = average_profit_diff.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_profit_diff.plot(kind='bar', color='skyblue')
plt.title('Average Profit Difference by Car Manufacturer')
plt.xlabel('Car Manufacturer')
plt.ylabel('Average Profit Difference')
plt.xticks(rotation=80)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

The bar chart above only demonstrate that most of the car brands can have a higher profit after being resold, revealing car prices does not only affected by time.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(diff['time_diff'], diff['profit_diff'], color='blue', alpha=0.7)
plt.title('Profit Difference vs. Time Difference Between Sales')
plt.xlabel('Time Difference (Days)')
plt.ylabel('Profit Difference')
plt.grid(True)
plt.tight_layout()
plt.show()

The graph above does not show an obvious relationship, supporting previous hypothesis that their is other factors affecting the price of car, so we are trying to find a relationship by clustering

## Clustering

The story here:
As shown in the first graph, the relationship between odometer, time, and profit do not have an obvious relationship, so we try to cluster the data to discover the relationship within those factors.
The elbow method is used here to find a suitable k-value for KMeans clustering, where the result indicates k = [2,3], we use 3 in this case.
The last graph shown the data after clustering with k = 3,where the red indicate most of the population that the profit won't be affect within large range of time and odometer value; and green indicate certain time within a range of odometer value, the profit will maintain as some price as well.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the data
ax.scatter(diff['time_diff'], diff['odom_diff'], diff['profit_diff'],
           c='blue', marker='o', alpha=0.7)

# Set labels and title
ax.set_xlabel('Time Difference (Days)')
ax.set_ylabel('Odometer Difference')
ax.set_zlabel('Profit Difference')
ax.set_title('Relationship between the Difference of Odometer, Time, and Profit after Reselling')

# Show plot
plt.show()

In [ ]:
from sklearn.cluster import KMeans
data = diff[['odom_diff', 'time_diff', 'profit_diff']]
distortions = []
k_range = range(1, 10)
for k in k_range:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(data)
    distortions.append(kmeans.inertia_) # Question: What does kmeans.inertia_ return?

plt.plot(k_range, distortions, 'bx-')

plt.title('The Elbow Method showing the optimal k')
plt.xlabel('k')
plt.ylabel('Distortion')

plt.show()

In [ ]:
clusters = KMeans(n_clusters=3)
clusters.fit(data)
def plot_kmeans(df, clusters):
    """
    generate a 3d plot given sklearn's kmeans implementation
    """
    colormap = {0: 'red', 1: 'green', 2: 'blue'}

    fig = plt.figure(figsize=(7, 10))
    ax = plt.axes(projection="3d")
    ax.scatter(df['time_diff'],
               df['odom_diff'],
               df['profit_diff'],
               c=[colormap.get(x) for x in clusters.labels_])

    ax.set_ylabel('odom_diff')
    ax.set_xlabel('time_diff')
    ax.set_zlabel('profit_diff')
    ax.set_title(f"k = {len(set(clusters.labels_))}")

    plt.show()
plot_kmeans(data, clusters)

## Make & Model

In [ ]:
year_counts = df['year'].value_counts().sort_index()
recent_years = year_counts[year_counts.index >= 2000]

plt.figure(figsize=(10, 6))
recent_years.plot(kind='bar')
plt.title('Popularity of Car Production Years')
plt.xlabel('Year')
plt.ylabel('Number of Cars')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
df['make'] = df['make'].str.lower()
make_counts = df['make'].value_counts().sort_values(ascending=False)

top_n = 15
top_makes = make_counts.head(top_n)

plt.figure(figsize=(12, 6))
top_makes.plot(kind='bar')
plt.title(f'Top {top_n} Most Popular Car Makes')
plt.xlabel('Make')
plt.ylabel('Number of Cars')
plt.xticks(rotation=45)

plt.gca().set_xticklabels([make.capitalize() for make in top_makes.index])

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
df['model'] = df['model'].str.lower()
model_counts = df['model'].value_counts().sort_values(ascending=False)

top_n = 15
top_models = model_counts.head(top_n)

plt.figure(figsize=(12, 6))
top_models.plot(kind='bar')
plt.title(f'Top {top_n} Most Popular Car Models')
plt.xlabel('Model')
plt.ylabel('Number of Cars')
plt.xticks(rotation=45)

plt.gca().set_xticklabels([model.capitalize() for model in top_models.index])

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()